In [1]:
import os 

In [2]:
%pwd

'c:\\code\\ML\\e-commerce\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\code\\ML\\e-commerce'

In [5]:
#config yaml
#config entity

In [6]:
#configuration

In [7]:
from e_commerce.utils.common import read_yaml , create_directories 
from e_commerce.constants import *
from e_commerce.entity.config_entity import DataIngestionconfig,PreProcessing,FeatureEngineeringconfig,EDA,outlier,cluster,model,prediction

class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionconfig(root_dir=config.root_dir,
                                                    source_url=config.source_url,
                                                    local_data_file=config.local_data_file)
        return data_ingestion_config
    
    def get_preprocessing_config(self)->PreProcessing:
        config = self.config.pre_processing
        create_directories([config.cleaned_data_save_path])
        create_directories([config.data_report])
        pre_processing_config = PreProcessing(data_path=config.data_path,cleaned_data_save_path=config.cleaned_data_save_path,data_report=config.data_report
                                              )
        return pre_processing_config
    
    def get_feature_engg_config(self)->FeatureEngineeringconfig:
        config = self.config.feature_engg
        create_directories([config.output_path])
        feature_engg_config = FeatureEngineeringconfig(data_path=config.data_path,output_path=config.output_path)
        return feature_engg_config 
    
    def get_eda_config(self)->EDA:
        config = self.config.EDA
        create_directories([config.report])
        eda_config = EDA(data_path=config.data_path,report=config.report)

        return eda_config
    
    def outlier_config(self)->outlier:
        config = self.config.outlier
        create_directories([config.output_path])
        create_directories([config.report])
        outlier_config = outlier(data_path=config.data_path,output_path=config.output_path,report=config.report)
        return outlier_config
    def cluster_config(self)->cluster:
        config = self.config.cluster
        params = self.params.cluster
        create_directories([config.cluster])
        create_directories([config.report])
        cluster_config = cluster(data_path=config.data_path,cluster=config.cluster,report=config.report,random_state= params.random_state)
        return cluster_config
    def get_model_config(self)->model:
        config = self.config.model
        params = self.params.model
        create_directories([config.models])
        create_directories([config.report])
        model_config = model(data_path=config.data_path,kl_path=config.kl_path,models=config.models,report=config.report,random_state=params.random_state)
        return model_config
    
    def prediction_config(self)->prediction:
        config = self.config.prediction
        prediction_config = prediction(model_path=config.model_path,scaler_path=config.scaler_path)
        return prediction_config
    

In [8]:
#component

In [9]:
import joblib
import pandas as pd

class prediction_component:
    def __init__(self,config:prediction):
        self.config = config
        self.model = joblib.load(self.config.model_path)
        self.scaler = joblib.load(self.config.scaler_path)
    def predict(self,input:pd.DataFrame):
        scaled_data = self.scaler.transform(input)
        predicted = self.model.predict(scaled_data)

        segment_map = {
            0: 'Loyal Customers',
            1: 'Churned Customers',
            2: 'Potential Loyalists'
        }
        label = segment_map.get(predicted[0],"Unknown")
        return label

        


        

In [16]:
#pipeline ----> app.py

In [11]:
from flask import Flask, render_template, request
import pandas as pd

In [15]:
app = Flask(__name__)

@app.route('/',methods = ['GET','POST'])

def index():
    if request.method == 'POST':
        try:
            features = [float(request.form.get("recency")),
                        float(request.form.get("frequency")),
                        float(request.form.get("monetary"))]
            feature_columns = ['recency','frequency','monetary']

            input_data = pd.DataFrame([features],columns=feature_columns)

            pred_config = ConfigurationManager().prediction_config()
            predictor = prediction_component(config=pred_config)
            result = predictor.predict(input=input_data)

            return render_template('index.html',prediction = result)
        except Exception as e:
            return render_template('index.html', error=str(e))
    return render_template('index.html')
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on
[2025-06-01 19:54:15,807: INFO: _internal: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.43.20:8000]
[2025-06-01 19:54:15,810: INFO: _internal: Press CTRL+C to quit]


[2025-06-01 19:54:20,604: INFO: _internal: 127.0.0.1 - - [01/Jun/2025 19:54:20] "GET / HTTP/1.1" 200 -]
[2025-06-01 19:54:33,269: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-01 19:54:33,281: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-01 19:54:33,296: INFO: common: created directory at: artifacts]


c:\code\ML\e-commerce\env310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


[2025-06-01 19:54:33,921: INFO: _internal: 127.0.0.1 - - [01/Jun/2025 19:54:33] "POST / HTTP/1.1" 200 -]
[2025-06-01 19:54:53,317: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-01 19:54:53,341: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-01 19:54:53,345: INFO: common: created directory at: artifacts]
[2025-06-01 19:54:53,391: INFO: _internal: 127.0.0.1 - - [01/Jun/2025 19:54:53] "POST / HTTP/1.1" 200 -]


c:\code\ML\e-commerce\env310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


[2025-06-01 19:55:44,852: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-01 19:55:44,889: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-01 19:55:44,889: INFO: common: created directory at: artifacts]
[2025-06-01 19:55:44,918: INFO: _internal: 127.0.0.1 - - [01/Jun/2025 19:55:44] "POST / HTTP/1.1" 200 -]


c:\code\ML\e-commerce\env310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
